In [ ]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import json
import requests
import pandas as pd
from bokeh.models.tools import HoverTool

output_notebook()

In [ ]:
colors_dict = {}
colors_dict['open'] = 'red'
colors_dict['close'] = 'blue'
colors_dict['adj_close'] = 'navy'

In [ ]:
def get_key(json_file='alpha_vantage_config.json',json_key='alpha_vantage_info'):
    with open(json_file) as f:
        data = json.load(f)
    
    user_values = data[json_key]
    return user_values['api_key']

In [ ]:
json_file = '/Users/adamgifford/Documents/DataScience/DataIncubator/Winter 2021/12_Day_Challenge/alpha_vantage_config.json'
api_key = get_key(json_file=json_file)

In [ ]:
appvars = {}
appvars['symbol'] = 'IBM'
appvars['open'] = 'Open'
appvars['close'] = 'Close'
# appvars['adj_close'] = 'Adjusted Close'
appvars['adj_close'] = None

In [ ]:
def get_url(dict_key, dict_val):
    base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
    adj_base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED'

    if dict_val is not None:
        if 'adj' in dict_key:
            return adj_base_url
        else:
            return base_url

In [ ]:
def fetch_response(symbol,api_key,url):
    url_search = '&symbol={}&apikey={}'.format(symbol, api_key)
    url = url + url_search
    return requests.get(url).json()

In [ ]:
dict_key = 'adj_close'
url = get_url(dict_key,appvars[dict_key])
url

In [ ]:
response = fetch_response('IBM',api_key,url)

In [ ]:
def make_df(response):
    df = pd.DataFrame.from_dict(response['Time Series (Daily)'], orient='index')
    df.rename(columns=lambda x: x[3:], inplace=True)
    df.index = pd.to_datetime(df.index)
    df.sort_index(ascending=False,inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'date'}, inplace=True)
    return df

In [ ]:
df = make_df(response)

In [ ]:
def create_title(appvars):
    title_str = ''
    for key,val in appvars.items():
        if key=='symbol':
            title_str = title_str + appvars['symbol'] + ' stock: '
        else:
            if appvars[key] is not None:
                title_str = title_str + appvars[key] + ', '

    title_str = title_str.split(',')
    title_str[-2] = '& ' + title_str[-2].strip()
    title_str = ', '.join(title_str)
    title_str = title_str[:-3]
    return title_str

In [ ]:
title_str = create_title(appvars)
title_str

In [ ]:
df.head()

In [ ]:
from datetime import date, timedelta
right = pd.to_datetime(date.today().strftime("%Y-%m-%d"))
d = date.today() - timedelta(days=30)
left = pd.to_datetime(d.strftime("%Y-%m-%d"))
left,right

In [ ]:
dur = right-left
int(dur.total_seconds() * 1e9)

In [ ]:
foo = df['date'].astype(int)

In [ ]:
1.609e12

In [ ]:
24*60*60*1000*1000*1000

In [ ]:
tips = [("Date",'@date{%F}')]

In [ ]:
tips.append(("Open",'@open'))
tips

In [ ]:
def make_line(df,handle,dict_val,color):
    if dict_val is not None:
        l = handle.line('date', dict_val.lower(), line_width=2, line_color=color, alpha = 0.5, legend_label=dict_val.lower(), source=df)
        ytip = (dict_val.title(), '@{' + dict_val.lower() + '}')
        return l, ytip
    
    return None,None

In [ ]:
TOOLS = 'box_zoom,reset'
# create a new plot (with a title) using figure
p = figure(x_axis_type="datetime", title=title_str, plot_height=350, plot_width=700, 
           x_axis_label='Date', y_axis_label='Price ($)', tools=TOOLS)

In [ ]:
from bokeh.models import CustomJS, DateSlider
from bokeh.layouts import column

tips = [("Date",'@date{%F}')]
handle_grabbed = False
for key,val in appvars.items():
    if key!='symbol':
        color = colors_dict[key]
        l, tip = make_line(df,p,val,color)
        if l is not None:
            tips.append(tip)
            if not handle_grabbed:
                rndr = l
                handle_grabbed = True            

# tips = [("Date",'@date{%F}'),("Open",'@open'),("Close",'@close'), ("Adj. Close", '@{adjusted close}')]
hover = HoverTool(tooltips=tips, mode='vline', point_policy='follow_mouse', show_arrow=False, renderers=[rndr], formatters = {'@date':'datetime'})
p.add_tools(hover)

callback = CustomJS(args=dict(x_range=p.x_range), code="""
    var right = cb_obj.value
    var left = right - 2592000000
    x_range.start = left
    x_range.end = right
""")
slider = DateSlider(start=left, end=right, value=right, step=86400000, title='End Date')
# slider.format('datetime')
slider.js_on_change('value', callback)

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

p.x_range.start = left
p.x_range.end = right
layout = column(slider, p)
show(layout)

In [ ]:
slider.format

In [ ]:
p2 = figure(x_axis_type="datetime", title=title_str, plot_height=350, plot_width=700)
l1 = p2.line(df.index, df['close'], line_width=2, line_color=colors_dict['close'], alpha = 0.5)
show(p2)

In [ ]:
response['Time Series (Daily)']

In [ ]:
response